In [38]:
# build transaction
from algosdk.future import transaction
from algosdk import constants
from algosdk.v2client import algod
from algosdk import account, mnemonic
import base64
import logging


In [39]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
file_handler = logging.FileHandler("../logs/new_account_info.log")
formatter = logging.Formatter(
    "%(message)s \n")
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

In [40]:
authorized_accounts = ["one", "two", "three"]
test_account = "three"
def check_availability():
    status = 0
    for i in range(len(authorized_accounts)):
        if(test_account == authorized_accounts[i]):
            status = 1
        else:
            status = 0
    return status

In [41]:
check_availability()

1

In [42]:
# create dummy accounts.

def generate_algorand_keypair():
    private_key, address = account.generate_account()
    passphrase = mnemonic.from_private_key(private_key)
    _account = {"passphrase":passphrase, "private_key":private_key, "address": address}
    return _account


In [43]:
def get_private_key_from_mnemonic(mn) :
    private_key = mnemonic.to_private_key(mn)
    return private_key

In [44]:

def get_client(addr, token):
    algod_client = algod.AlgodClient(token, addr)
    return algod_client

In [45]:
def fund_account(client, sender, receiver, amount):
    params = client.suggested_params()
    params.flat_fee = True
    params.fee = constants.MIN_TXN_FEE 
    note = "Hello World".encode()
    unsigned_txn = transaction.PaymentTxn(sender, params, receiver, amount, None, note)

    return unsigned_txn, params.fee

In [46]:
# sign transaction

def sign_transaction(private_key, unsigned_txn):
    signed_txn = unsigned_txn.sign(private_key)

    return signed_txn

In [47]:
def send_trans(client, signed_txn):
    txid = client.send_transaction(signed_txn)

    # # wait for confirmation 
    try:
        confirmed_txn = transaction.wait_for_confirmation(client, txid, 4)  
        print("Successfully sent transaction with txID: {}".format(txid))
        return [txid, confirmed_txn]
    except Exception as err:
        print(err)
        print("Transaction with txID failed: {}".format(txid))
        return [txid, "failed"]
    

In [48]:
def create_funded_accounts(num_accounts, client, sender, amount):
    for i in range(num_accounts):
        account = generate_algorand_keypair()
        # fund created account
        account_info = client.account_info(account["address"])
        transaction, fee = fund_account(client, sender["address"], account["address"], amount)
        # sign transaction
        signed_txn = sign_transaction(sender["private_key"], transaction)
        # send transaction
        info = send_trans(client, signed_txn)
        if(info[1] != "failed"):
            confirmed_txn = info[1]
            logger.info("*"*50)
            logger.info("*"*50)
            logger.info(account)
            logger.info("*"*50)
            logger.info("Decoded note: {}".format(base64.b64decode(
                confirmed_txn["txn"]["txn"]["note"]).decode()))
            
            logger.info("Starting Account balance: {} microAlgos".format(account_info.get('amount')) )
            logger.info("Amount transfered: {} microAlgos".format(amount) )    
            logger.info("Fee: {} microAlgos".format(fee) ) 
            account_info = client.account_info(account["address"])
            logger.info("Final Account balance: {} microAlgos".format(account_info.get('amount')) + "\n")
            logger.info("\n")
            
        

In [50]:
#initilize client
algod_address = "http://localhost:4001"
algod_token = "aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa"
client = get_client(algod_address, algod_token)
num_accounts = 3
sender_addr = "L6ZS626IHSZN7SB6B2WFVW4G6EL2VNKGPP6KQB2FMOQOCRLAZFG3C5BIIY"
sender_pass = "dove tomato solid light purpose shrug shaft blade scrub ten peace bulk \
                net dynamic broken exotic dog knife method august goat drive recycle abandon limb"
sender_private = get_private_key_from_mnemonic(sender_pass)
sender = {"passphrase":sender_pass, "private_key":sender_private, "address": sender_addr}
amount = 100000

# create funded accounts
create_funded_accounts(num_accounts, client, sender, amount)


Successfully sent transaction with txID: HHHMB6GNXLQHUAVTXMFOAFUCBSDIK22GBQLMT7LYAYL2H54BHQVA
Successfully sent transaction with txID: TS53BWE7YJPWEXS37PPBSUEGRL2Z34LJVD3M4BPJTZ7KIA72C55A
Successfully sent transaction with txID: S4LBDNIPA74UTUBVBAXESOVTNRZT3CYNHEBVY7GEUGJECRZ3BHCA
